여기에 블렌딩 관련 작업

In [1]:
# from desktop w/ gpu
import tensorflow as tf
import numpy as np
import cv2
import time
import os
from enum import Enum
import keras_cv
import keras
import matplotlib.pyplot as plt
from utils import *

print(keras_cv.__version__)
print(keras.__version__)

print(tf.config.list_physical_devices('GPU'))
print(cv2.__version__)
print(cv2.cuda.getCudaEnabledDeviceCount())
print(tf.__version__)

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices() )
print(cv2.getBuildInformation())

%load_ext autoreload
%autoreload 2



2025-05-07 18:37:44.403265: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-07 18:37:44.600140: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-07 18:37:44.600180: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-07 18:37:44.631347: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-07 18:37:44.701718: I tensorflow/core/platform/cpu_feature_guar

Using TensorFlow backend
0.9.0
2.15.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
4.12.0-dev
1
2.15.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12851407868879145965
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10177478656
locality {
  bus_id: 1
  links {
  }
}
incarnation: 409005341030696435
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]

General configuration for OpenCV 4.12.0-dev =====================================
  Version control:               4.11.0-356-g325e59bd4c

  Extra modules:
    Location (extra):            /home/primarina314/opencv_contrib/modules
    Version control (extra):     4.11.0-42-gacd145f4

  Platform:
    Timestamp:                   2025-04-28T09:41:11Z
    Host:                        Linux 5.15.167.4-microsoft-standard-WSL2 x86_64
    CMake:                  

2025-05-07 18:37:47.174805: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-07 18:37:47.217167: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-07 18:37:47.217204: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-07 18:37:47.237654: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-07 18:37:47.237718: I external/local_xla/xla/stream_executor

In [1]:
# from laptop w/o gpu
import numpy as np
import cv2
import time
import os
from enum import Enum
import matplotlib.pyplot as plt
from collections import defaultdict
from PIL import Image
from utils import *

print(cv2.__version__)

%load_ext autoreload
%autoreload 2

4.12.0-dev


# 경계 블렌딩

우선 아래 방법 적용하지 않은 상태로 구현

- 원본 파트에서 경계에서 떨어진 거리에 따라 블렌딩 비율 다르게
- 경계에 가까울수록 downscaled->upscaled 이미지 비율을 높게 잡고
- 경계에서 멀어질수록 original part 이미지 비율을 높게 설정하여 블렌딩
- 일정 거리 이상으로 멀어지면 100% 원본파트
- 선형, sigmoid 등 함수로 interpolation
- roi 선택 시에 각 좌표를 모두 저장 -> 좌표를 바탕으로 거리 계산 -> 곡선형태로 저장할 수도 있으므로 더 광범위하게 적용가능한 findContour 함수로 처리하는게 나을듯

In [2]:
class blend_mode(Enum):
    alpha = 0
    poisson = 1
    laplacian = 2
    grad = 3
    dl = 4

def blending_two_images(src, target, mask, mode = blend_mode.poisson, center = (0, 0)):
    match mode:
        case blend_mode.alpha:
            result = cv2.addWeighted(src, 0.7, target, 0.3, 0, mask=mask)
        case blend_mode.poisson:
            result = cv2.seamlessClone(
                src, target, mask, center, cv2.MIXED_CLONE
            )
        case blend_mode.laplacian:
            # laplacian dist
            return
        case blend_mode.grad:
            # gradient
            return
        case blend_mode.dl:
            # dl model
            return

    return result

In [3]:
img_path = 'Lenna_(test_image).png'

# down->upscaled 된거 path
# original part path

output_path = 'blending-test'

# 블렌딩

In [78]:
roi, _ = select_polygon_roi(img_path)
mask = np.all(roi != [0, 0, 0], axis=2)
mask_uint8 = mask.astype(np.uint8) * 255

In [80]:

def point_line_distance(point, line_start, line_end):
    """
    점과 선분 사이의 최소 수직거리 계산
    point: (x, y) 튜플
    line_start, line_end: 선분의 두 끝점 (x, y) 튜플
    """
    px, py = point
    x1, y1 = line_start
    x2, y2 = line_end

    # 선분 벡터
    line_vec = np.array([x2 - x1, y2 - y1])
    # 점과 선분 시작점 벡터
    point_vec = np.array([px - x1, py - y1])

    line_len_sq = np.dot(line_vec, line_vec)
    if line_len_sq == 0:
        # 선분이 점인 경우
        return np.linalg.norm(point_vec)

    # 점에서 선분으로의 투영 비율
    t = np.dot(point_vec, line_vec) / line_len_sq

    if t < 0.0:
        # 투영점이 선분 시작점 밖에 있는 경우
        closest_point = np.array([x1, y1])
    elif t > 1.0:
        # 투영점이 선분 끝점 밖에 있는 경우
        closest_point = np.array([x2, y2])
    else:
        # 투영점이 선분 위에 있는 경우
        closest_point = np.array([x1, y1]) + t * line_vec

    # 점과 가장 가까운 점 사이 거리
    dist = np.linalg.norm(np.array([px, py]) - closest_point)
    return dist

def distance_to_polygon_edge(mask, point):
    """
    mask: 2D numpy array, 다각형 영역이 1(또는 255), 배경이 0인 바이너리 마스크
    point: (x, y) 튜플, 기준점 좌표

    반환값: point에서 다각형 경계까지의 최소 수직거리 (float)
    """
    # 마스크를 바이너리(0 또는 255)로 변환
    mask_bin = (mask > 0).astype(np.uint8) * 255

    # 다각형 외곽선(경계) 검출
    contours, _ = cv2.findContours(mask_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return None  # 다각형이 없는 경우

    min_dist = float('inf')
    for contour in contours:
        pts = contour.reshape(-1, 2)
        n = len(pts)
        for i in range(n):
            start = tuple(pts[i])
            end = tuple(pts[(i + 1) % n])
            dist = point_line_distance(point, start, end)
            if dist < min_dist:
                min_dist = dist

    return min_dist

# 사용 예시
# mask = np.zeros((200, 200), dtype=np.uint8)
# polygon = np.array([[50, 50], [150, 50], [150, 150], [50, 150]])
# cv2.fillPoly(mask, [polygon], 255)

# point = (100, 100)  # 예시 기준점
# distance = distance_to_polygon_edge(mask, point)
# print(f"Distance from point {point} to polygon edge: {distance}")


In [91]:
# dist = distance_to_polygon_edge(mask, (100, 100))
# print(dist)

def check_dist(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        dist = distance_to_polygon_edge(mask, (x, y))
        print(f"distance from ({x}, {y}) to edge: {dist}")

cv2.namedWindow("test")
cv2.setMouseCallback("test", check_dist)

cv2.imshow('test', mask_uint8)

clicking = True
while True:
    key = cv2.waitKey(1)
    if key == 27:
        break

cv2.destroyAllWindows()

distance from (40, 51) to edge: 0.0
distance from (21, 172) to edge: 0.7071067811865476
distance from (74, 146) to edge: 36.40054944640259
distance from (59, 79) to edge: 33.421549934136806
distance from (60, 33) to edge: 2.1213203435596424
distance from (30, 54) to edge: 3.605551275463989
distance from (85, 19) to edge: 0.7071067811865476
distance from (59, 34) to edge: 2.0
distance from (43, 48) to edge: 0.7071067811865476
distance from (62, 40) to edge: 4.47213595499958
distance from (63, 38) to edge: 3.5355339059327378
distance from (63, 33) to edge: 0.0
distance from (61, 34) to edge: 0.7071067811865476
distance from (61, 113) to edge: 57.14017850864661
distance from (75, 170) to edge: 14.560219778561036
distance from (94, 108) to edge: 31.144823004794873
distance from (3, 189) to edge: 24.698178070456937
distance from (14, 179) to edge: 10.0
distance from (38, 140) to edge: 31.400636936215164
distance from (118, 173) to edge: 13.892443989449804
distance from (25, 32) to edge: 24.

In [70]:
cv2.imshow('test', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [65]:
print(mask)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:



mrs3_compress(img_path, output_path, 4, ROI_mode.Polygon)


original file: 473831
downscaled filesize: 35816
roi filesize: 27504
config filesize: 97
compression ratio: 0.1338388581582885


In [6]:
mrs3_restore('qwergfhfghgfhgfg', MRS3_mode.edsr, 'qwergfhfghgfhgfg/restored.png')

In [7]:
def distance_to_polygon_edge(mask, point):
    """
    mask: 2D numpy array, polygon 영역이 1(또는 255), 배경이 0인 바이너리 마스크
    point: (x, y) 튜플, 기준점 좌표

    반환값: point에서 다각형 경계까지의 최단 거리 (float)
    """
    # 마스크를 바이너리(0 또는 255)로 변환
    mask_bin = (mask > 0).astype(np.uint8) * 255

    # 다각형 외곽선(경계) 검출
    contours, _ = cv2.findContours(mask_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return None  # 다각형이 없는 경우

    # 모든 경계점에 대해 거리 계산, 최솟값 반환
    min_dist = float('inf')
    for contour in contours:
        for pt in contour:
            dist = np.linalg.norm(np.array(pt[0]) - np.array(point))
            if dist < min_dist:
                min_dist = dist

    return min_dist

# 사용 예시
# mask: 다각형이 그려진 바이너리 마스크 (예: 200x200, 가운데 사각형)
# mask = np.zeros((200, 200), dtype=np.uint8)
# polygon = np.array([[50, 50], [150, 50], [150, 150], [50, 150]])
# cv2.fillPoly(mask, [polygon], 255)

# point = (100, 100)  # 예시 기준점

# distance = distance_to_polygon_edge(mask, point)
# print(f"Distance from point {point} to polygon edge: {distance}")


In [92]:
# TODO: 경계 interpolation 한계 설정